<a href="https://colab.research.google.com/github/Mbishu2002/ant/blob/main/ant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np


data = pd.read_csv("your_dataset.csv")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class CustomDataset():
    def __init__(self, data):
        self.data = data
        self.tokenized_data = []
        for text in data['quantitative_analysis']:
            tokenized_text = tokenizer.encode(text, add_special_tokens=True)
            self.tokenized_data.append(tokenized_text)

custom_dataset = CustomDataset(data)


model = BertModel.from_pretrained('bert-base-uncased')


def extract_bert_embeddings(data, model):
    embeddings = []
    with torch.no_grad():
        for text in data:
            inputs = torch.tensor(text).unsqueeze(0)
            outputs = model(inputs)
            last_hidden_states = outputs.last_hidden_state
            cls_embedding = last_hidden_states[:, 0, :]
            embeddings.append(cls_embedding.squeeze().numpy())
    return embeddings


embeddings = extract_bert_embeddings(custom_dataset.tokenized_data, model)


pca = PCA(n_components=50)
reduced_embeddings = pca.fit_transform(embeddings)


kmeans = KMeans(n_clusters=5)
clusters = kmeans.fit_predict(reduced_embeddings)


for cluster_idx in range(kmeans.n_clusters):
    cluster_data = data.iloc[np.where(clusters == cluster_idx)]

